# It isn't obvious from the start how to use BERT.

Hmm here's some repo for Chinese NLP models https://github.com/lonePatient/awesome-pretrained-chinese-nlp-models

Hmm let's just the pretrained one on huggingface https://huggingface.co/bert-base-chinese

In [1]:
import torch
import transformers
from transformers import BertForMaskedLM

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

PRETRAINED_MODEL_NAME = "bert-base-chinese"

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# hmmm looking at some tutorial here

Source: https://www.heywhale.com/mw/project/609f609f06b942001794ff44

- [CLS]：在做分类任务时其最后一层的repr. 会被视为整个输入序列的repr.
- [SEP]：有两个句子的文本会被串接成一个输入序列，并在两句之间插入这个token 以做区隔
- [UNK]：没出现在BERT 字典里头的字会被这个token 取代
- [PAD]：zero padding 遮罩，将长度不一的输入序列补齐方便做batch 运算
- [MASK]：未知遮罩，仅在预训练阶段会用到

In [1]:
vocab = tokenizer.vocab
print("vocab size: ", len(vocab))

NameError: name 'tokenizer' is not defined

In [17]:
text = "[CLS] 等到潮水 [MASK] 了，就知道谁沒穿裤子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens[:10], '...')
print(ids[:10], '...')


[CLS] 等到潮水 [MASK] 了，就知道谁沒穿裤子。
['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
[101, 5023, 1168, 4060, 3717, 103, 749, 8024, 2218, 4761] ...


In [20]:
ids

[101,
 5023,
 1168,
 4060,
 3717,
 103,
 749,
 8024,
 2218,
 4761,
 6887,
 6443,
 3760,
 4959,
 6175,
 2094,
 511]

In [23]:
from transformers import BertForMaskedLM
# 除了 tokens 以外我們還需要辨別句子的 segment ids
tokens_tensor = torch.tensor([ids])  # (1, seq_len)
segments_tensors = torch.zeros_like(tokens_tensor)  # (1, seq_len)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)

# 使用 masked LM 估計 [MASK] 位置所代表的實際 token 
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensors)
    predictions = outputs[0]
    # (1, seq_len, num_hidden_units)
del maskedLM_model

# 將 [MASK] 位置的機率分佈取 top k 最有可能的 tokens 出來
masked_index = 5
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())

# 顯示 top k 可能的字。一般我們就是取 top 1 当做预测值
print("輸入 tokens ：", tokens[:10], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    tokens[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens[:10]), '...')


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


輸入 tokens ： ['[CLS]', '等', '到', '潮', '水', '[MASK]', '了', '，', '就', '知'] ...
--------------------------------------------------
Top 1 (65%)：['[CLS]', '等', '到', '潮', '水', '来', '了', '，', '就', '知'] ...
Top 2 ( 4%)：['[CLS]', '等', '到', '潮', '水', '过', '了', '，', '就', '知'] ...
Top 3 ( 4%)：['[CLS]', '等', '到', '潮', '水', '干', '了', '，', '就', '知'] ...


In [25]:
from transformers import AutoTokenizer, AutoModel
from bertviz import model_view

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese", output_attentions=True)
inputs = tokenizer.encode("[CLS] 等到潮水 [MASK] 了，就知道谁沒穿裤子。", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
model_view(attention, tokens)

ModuleNotFoundError: No module named 'bertviz'

In [7]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [8]:
inputs = tokenizer(['这是什么让我试试切词。你好', '这是什么让我试试切词。你好了卡就是打开了放假啊可是你的肌肤'], return_tensors="pt", padding=True)
outputs = model(**inputs)

In [9]:
inputs

{'input_ids': tensor([[ 101, 6821, 3221,  784,  720, 6375, 2769, 6407, 6407, 1147, 6404,  511,
          872, 1962,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0],
        [ 101, 6821, 3221,  784,  720, 6375, 2769, 6407, 6407, 1147, 6404,  511,
          872, 1962,  749, 1305, 2218, 3221, 2802, 2458,  749, 3123,  969, 1557,
         1377, 3221,  872, 4638, 5491, 5502,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}

In [10]:
outputs

MaskedLMOutput(loss=None, logits=tensor([[[ -7.9119,  -7.8119,  -7.8056,  ...,  -6.5529,  -6.1713,  -6.4862],
         [ -8.2322,  -8.0884,  -8.0818,  ...,  -6.7162,  -6.2229,  -6.5252],
         [-16.1794, -16.3523, -18.0674,  ...,  -8.4667,  -4.2078,  -5.2893],
         ...,
         [ -7.7972,  -7.5529,  -7.6924,  ...,  -4.1089,  -2.6128,  -3.1202],
         [ -7.9204,  -7.5319,  -7.7652,  ...,  -4.1665,  -2.4351,  -3.1046],
         [ -7.9449,  -7.6439,  -7.8875,  ...,  -4.2426,  -2.6454,  -3.0096]],

        [[ -8.0285,  -7.9644,  -7.9966,  ...,  -6.9183,  -6.6847,  -7.0758],
         [ -8.1193,  -7.9995,  -7.9447,  ...,  -6.7129,  -6.3368,  -6.5415],
         [-16.6799, -16.4066, -18.4110,  ...,  -9.9241,  -4.0629,  -5.6410],
         ...,
         [-13.4532, -13.5211, -12.6752,  ...,  -9.8302,  -4.2376, -10.3354],
         [-10.6056, -10.2510, -10.3465,  ...,  -8.8303,  -3.1233,  -7.3241],
         [-10.3576, -10.5163, -10.6644,  ...,  -8.2772,  -5.6076,  -8.5861]]],
       grad

In [11]:
outputs.logits.shape

torch.Size([2, 31, 21128])

In [13]:
model.get_output_embeddings()

Linear(in_features=768, out_features=21128, bias=True)

# Okay not let's figure out how to use the outputs

Sources: 

1. done https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1
2. https://towardsdatascience.com/bert-text-classification-in-a-different-language-6af54930f9cb
3. https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
4. https://towardsdatascience.com/how-bert-determines-search-relevance-2a67a1575ac4
5. https://bergum.medium.com/how-not-to-use-bert-for-search-ranking-4586716428d9


# Now try to do some document ranking.

Emm but no idea how to do that... No worries let's read some articles

1. https://medium.com/@papai143/information-retrieval-with-document-re-ranking-with-bert-and-bm25-7c29d738df73
2. https://medium.com/nerd-for-tech/bert-qe-contextualized-query-expansion-for-document-re-ranking-4f0f421840b9

# Try sentence transformer?

In [10]:
from sentence_transformers import SentenceTransformer
from pprint import pprint

In [2]:
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

In [4]:
sentence_transformer_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

Downloading: 100%|██████████| 1.58M/1.58M [00:00<00:00, 14.1MB/s]


In [8]:
sentence_embeddings = sentence_transformer_model.encode(sentences)

In [14]:
#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-3.21109705e-02 -2.91681308e-02 -4.93781529e-02  5.35542425e-03
  5.79125322e-02 -7.43310079e-02 -5.69196492e-02 -2.62950640e-02
 -2.77417712e-05 -4.68358630e-03 -5.46516441e-02 -9.72127309e-05
 -2.39549577e-03 -4.00913805e-02 -3.10327299e-02 -3.37153822e-02
 -7.05639832e-03  3.61287929e-02  2.57309480e-03 -5.69516756e-02
  5.62542081e-02  1.25679923e-02  1.77145321e-02 -3.04215793e-02
 -2.37881746e-02 -2.86213458e-02  1.87028330e-02  1.17944861e-02
  1.01970527e-02  1.55350352e-02  5.70734702e-02  4.81183529e-02
 -3.68592367e-02  1.59383602e-02 -6.20532595e-02  1.36246728e-02
 -2.07147878e-02  2.28246786e-02  3.50557789e-02  6.38704654e-03
 -2.75127441e-02  7.06582665e-02  3.77225950e-02 -3.34584806e-03
  5.38096391e-02  6.05076626e-02  6.33056387e-02  9.52539325e-04
 -1.29994014e-02 -2.43680999e-02 -6.48297071e-02  2.89445352e-02
 -1.87204557e-03  9.60670318e-03 -1.06688216e-02 -8.95597041e-03
 -5.76722